In [ ]:
!pip install google-cloud-aiplatform

In [33]:
from google.oauth2 import service_account
from google.cloud import aiplatform
import os

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "your-service-acct-cred.json"

In [98]:
from typing import Dict, List, Union

from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value


def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "asia-southeast1",
    api_endpoint: str = "asia-southeast1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if isinstance(instances, list) else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    return response

In [99]:
instances = [
    {
        "@requestFormat": "chatCompletions",
        "messages": [
            {
                "role": "user",
                "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": "https://static.ticketmaster.sg/images/activity/25sg_jackycheung_2d95b62dbb846d6870cce7bce52cb42a.jpg"
                            }
                        },
                        {
                            "type": "text",
                            "text": "Extract the text from the above document as if you were reading it naturally. Return the tables in html format. Return the equations in LaTeX representation. If there is an image in the document and image caption is not present, add a small description of the image inside the <img></img> tag; otherwise, add the image caption inside <img></img>. Watermarks should be wrapped in brackets. Ex: <watermark>OFFICIAL COPY</watermark>. Page numbers should be wrapped in brackets. Ex: <page_number>14</page_number> or <page_number>9/22</page_number>. Prefer using ☐ and ☑ for check boxes."
                        }
                    ]
            }
        ],
        "max_tokens": 1024,
        "temperature": 0.6,
        "top_p": 1.0,
        "top_k": -1
    }
]

In [ ]:
response = predict_custom_trained_model_sample(
    project="your-proj-id",
    endpoint_id="your-endpoint-id",
    instances=instances
)

In [101]:
type(response)

google.cloud.aiplatform_v1.types.prediction_service.PredictResponse

In [102]:
from google.protobuf.json_format import MessageToDict
response_dict = MessageToDict(response._pb) # only this works...

In [104]:
content = response_dict["predictions"][0][0]["message"]["content"]

In [105]:
content

'KRISFLYER 呈現\nSINGAPORE AIRLINES GROUP\n\nJACKY 張學友 60+\nJC CHEUNG CONCERT TOUR\n\n巡迴演唱會 新加坡 SINGAPORE\n\n2025.11.21/22/23 | 8PM | SINGAPORE INDOOR STADIUM 新加坡室内体育馆\n\n主办单位 ORGANIZER\nHorizon International Pte Ltd\n\n银行卡伙伴 CARD PARTNER OCBC\n\n主要赞助 MAIN SPONSOR Geneco\nPowered by YTL Property\n\n媒体伙伴 MEDIA PARTNERS\n联合早报\nLINGIANG ZHANGBO\n\n<watermark>JDS JDS ENTERTAINMENT 全球演藝集團</watermark>\n\n<img>A man in a white suit with a bow tie, surrounded by underwater-like effects.</img>\n<img>一个男人穿着白色西装，打着领结，周围有水下效果。</img>\n<img>A man in a white suit with a bow tie, surrounded by underwater-like effects.</img>\n<img>一个男人穿着白色西装，打着领结，周围有水下效果。</img>'